In [ ]:
import re
from nltk.stem import WordNetLemmatizer
import itertools
wordnet_lemmatizer = WordNetLemmatizer()
from tqdm.notebook import tqdm
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from string import ascii_lowercase
from functools import reduce
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import numpy as np

In [ ]:
with open('../params.json', 'r') as f:
    params = json.load(f)

max_length = params['max_length']
padding_type = params['padding_type']
vocab_size = params['vocab_size']
embedding_dim = params['embedding_dim']
trunc_type = params['trunc_type']
oov_tok = params['oov_tok']

#### RE:

In [ ]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],
    'america':
        [
            'amerika'
        ],
    ' adolf ':
        [
            'adolf'
        ],

    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}

#### Rest:

In [ ]:
train = pd.read_csv("../kaggle_data/train_x.csv").fillna('')
val = pd.read_csv("../kaggle_data/val_x.csv").fillna('')
test = pd.read_csv("../kaggle_data/test_x.csv").fillna('')


In [ ]:
train_y_all=pd.read_csv('../kaggle_data/train_y.csv')
val_y_all=pd.read_csv('../kaggle_data/val_y.csv')

In [ ]:
def preprocess_text(input_text, convert_to_lower=True, remove_patterns=True, remove_repeats=True):
    # Convert to lower case
    if convert_to_lower:
        input_text = input_text.lower()

    # Replace patterns
    if remove_patterns and RE_PATTERNS:
        for replacement, patterns in RE_PATTERNS.items():
            for pattern in patterns:
                input_text = input_text.replace(pattern, replacement)

    # Remove repeated characters
    if remove_repeats:
        input_text = re.sub(r'(.)\1{2,}', r'\1', input_text)

    # Replace newline characters with spaces
    input_text = input_text.replace("\n", " ")

    # Remove non-alphanumeric characters
    input_text = re.sub(r'[^\w\s]', ' ', input_text)

    # Remove numbers
    input_text = re.sub(r'\d', '', input_text)

    # Replace multiple spaces with a single space
    input_text = re.sub(r'\s+', ' ', input_text)

    # Remove non-ASCII characters
    input_text = re.sub(r'([^\x00-\x7F])+', ' ', input_text)

    return input_text.strip()  # Remove leading/trailing spaces

In [ ]:
train['string']=train.apply(lambda x: preprocess_text(x['string']),axis=1)
val['string']=val.apply(lambda x: preprocess_text(x['string']),axis=1)
test['string']=test.apply(lambda x: preprocess_text(x['string']),axis=1)

In [ ]:
def lemmatize_text(input_text, apply_lemmatization=True):
    # Initialize the lemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()

    # If lemmatization is not required, return the original text
    if not apply_lemmatization:
        return input_text

    # Split the text into words
    words = input_text.split()

    # Lemmatize each word
    lemmatized_words = []
    for word in words:
        noun = wordnet_lemmatizer.lemmatize(word, pos="n")
        verb = wordnet_lemmatizer.lemmatize(noun, pos="v")
        adjective = wordnet_lemmatizer.lemmatize(verb, pos="a")
        adverb = wordnet_lemmatizer.lemmatize(adjective, pos="r")
        lemmatized_words.append(adverb)

    # Join the lemmatized words back into a string
    lemmatized_text = " ".join(lemmatized_words)

    return lemmatized_text.strip()

In [ ]:
train['string']=train.apply(lambda x: lemmatize_text(x['string']),axis=1)
val['string']=val.apply(lambda x: lemmatize_text(x['string']),axis=1)
test['string']=test.apply(lambda x: lemmatize_text(x['string']),axis=1)

In [ ]:
def generate_all_strings():
    """
    Generate all possible strings of increasing length using the ASCII lowercase letters.
    """
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)

# List to hold all generated strings
generated_strings = []
for s in generate_all_strings():
    generated_strings.append(s)
    if s == 'zz':
        break

# List of words to remove from the generated strings
words_to_remove = ['i', 'a', 'am', 'an', 'as', 'at', 'be', 'by', 'do', 'go', 'he', 'hi', 'if', 'is', 'in', 'me', 'my', 'no', 'of', 'on', 'or', 'ok', 'so', 'to', 'up', 'us', 'we']

# Remove the specified words from the list of generated strings
for word in words_to_remove:
    if word in generated_strings:
        generated_strings.remove(word)

# Convert the list of generated strings to a set to remove any duplicates
stopword_set = set(generated_strings)

In [ ]:
lemmatized_train_data=train['string'].tolist()
lemmatized_val_data=val['string'].tolist()
lemmatized_test_data=test['string'].tolist()

In [ ]:
def find_non_stopwords(data, search_stop=True):
    """
    Function that reads the entire dataset and adds words that are not already present in stopword_set into a list of potential_stopwords.
    """
    output = ""
    if search_stop:
        data = data.split(" ")
        for word in data:
            if word not in stopword_set:
                output = output + " " + word 
    else:
        output = data
    return str(output.strip())

# Calling the find_non_stopwords function, and saving words into a list (potential_stopwords).
potential_stopwords = []
for line in tqdm(lemmatized_train_data, total=len(lemmatized_train_data)): 
    potential_stopwords.append(find_non_stopwords(line))

# Functions to combine all the sentences present in potential_stopwords into 4 different strings.
# Each function handles a different segment of the data, this facilitates faster concatenation of sentences. 
def combine_strings(stopwords, start, end):
    final_string = ""
    for item in range(start, end):
        final_string = final_string + " " + stopwords[item]
    return final_string

# Combining the potential stopwords into four different strings
total_string_potential_a = combine_strings(potential_stopwords, 0, 39893)
total_string_potential_b = combine_strings(potential_stopwords, 39893, 79785)
total_string_potential_c = combine_strings(potential_stopwords, 79785, 119678)
total_string_potential_d = combine_strings(potential_stopwords, 119678, 159571)

def count_words(str):
    """
    Function to count the number of words in a string and save it in a dictionary.
    """
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

# Counting the number of words in each of the 4 strings
total_string_potential_a_dict = count_words(total_string_potential_a)
total_string_potential_b_dict = count_words(total_string_potential_b)
total_string_potential_c_dict = count_words(total_string_potential_c)
total_string_potential_d_dict = count_words(total_string_potential_d)

# Converting Dictionaries to Dataframes.
def dict_to_df(dict):
    return pd.DataFrame(list(dict.items()),columns = ['Word','Count'])

total_string_potential_a_df = dict_to_df(total_string_potential_a_dict)
total_string_potential_b_df = dict_to_df(total_string_potential_b_dict)
total_string_potential_c_df = dict_to_df(total_string_potential_c_dict)
total_string_potential_d_df = dict_to_df(total_string_potential_d_dict)

# Getting Dataframe output in descending order.
def get_top50(df):
    return df.sort_values(by=['Count'],ascending=False).head(50)

top50_potential_stopwords_a = get_top50(total_string_potential_a_df)

In [ ]:
def find_non_stopwords(data, search_stop=True):
    """
    Function that reads the entire dataset and adds words that are not already present in stopword_set into a list of potential_stopwords.
    """
    output = ""
    if search_stop:
        data = data.split(" ")
        for word in data:
            if word not in stopword_set:
                output = output + " " + word 
    else:
        output = data
    return str(output.strip())

# Calling the find_non_stopwords function, and saving words into a list (potential_stopwords).
potential_stopwords = []
for line in tqdm(lemmatized_train_data, total=len(lemmatized_train_data)): 
    potential_stopwords.append(find_non_stopwords(line))

# Functions to combine all the sentences present in potential_stopwords into 4 different strings.
# Each function handles a different segment of the data, this facilitates faster concatenation of sentences. 
def combine_strings(stopwords, start, end):
    final_string = ""
    for item in range(start, end):
        final_string = final_string + " " + stopwords[item]
    return final_string

# Combining the potential stopwords into four different strings
total_string_potential_a = combine_strings(potential_stopwords, 0, 39893)
total_string_potential_b = combine_strings(potential_stopwords, 39893, 79785)
total_string_potential_c = combine_strings(potential_stopwords, 79785, 119678)
total_string_potential_d = combine_strings(potential_stopwords, 119678, 159571)

def count_words(str):
    """
    Function to count the number of words in a string and save it in a dictionary.
    """
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

# Counting the number of words in each of the 4 strings
total_string_potential_a_dict = count_words(total_string_potential_a)
total_string_potential_b_dict = count_words(total_string_potential_b)
total_string_potential_c_dict = count_words(total_string_potential_c)
total_string_potential_d_dict = count_words(total_string_potential_d)

# Converting Dictionaries to Dataframes.
def dict_to_df(dict):
    return pd.DataFrame(list(dict.items()),columns = ['Word','Count'])

total_string_potential_a_df = dict_to_df(total_string_potential_a_dict)
total_string_potential_b_df = dict_to_df(total_string_potential_b_dict)
total_string_potential_c_df = dict_to_df(total_string_potential_c_dict)
total_string_potential_d_df = dict_to_df(total_string_potential_d_dict)

# Getting Dataframe output in descending order.
def get_top50(df):
    return df.sort_values(by=['Count'],ascending=False).head(50)

top50_potential_stopwords_a = get_top50(total_string_potential_a_df)
top50_potential_stopwords_b = get_top50(total_string_potential_b_df)
top50_potential_stopwords_c = get_top50(total_string_potential_c_df)
top50_potential_stopwords_d = get_top50(total_string_potential_d_df)

# Looking for common terms in all top 50 dataframes.
common_potential_stopwords = list(reduce(set.intersection, map(set, [top50_potential_stopwords_a.Word, top50_potential_stopwords_b.Word, top50_potential_stopwords_c.Word, top50_potential_stopwords_d.Word])))

# Retaining certain words and removing others from the above list.
retained_stopwords = ['editor', 'reference', 'thank', 'work','find', 'good', 'know', 'like', 'look', 'thing', 'want', 'time', 'list', 'section','wikipedia', 'doe', 'add','new', 'try', 'think', 'write','use', 'user', 'way', 'page']

# Adding above retrieved words into the stopwords set.
for word in retained_stopwords:
    stopword_set.add(word)

In [ ]:
max_features=vocab_size
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train['string']))
list_tokenized_train = tokenizer.texts_to_sequences(train['string'])
list_tokenized_val = tokenizer.texts_to_sequences(val['string'])
list_tokenized_test = tokenizer.texts_to_sequences(test['string'])

In [ ]:
word_counts = tokenizer.word_counts
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
top_n_words = vocab_size - 1 

top_words_indices = {word: index for index, (word, _) in enumerate(sorted_word_counts[:top_n_words])}
filtered_word_index = {word: index for word, index in tokenizer.word_index.items() if word in top_words_indices}

print("Filtered Word Index:", len(filtered_word_index))

In [ ]:
maxpadlen = max_length
train_padded=pad_sequences(list_tokenized_train, maxlen=maxpadlen, padding = 'post')
val_padded=pad_sequences(list_tokenized_val, maxlen=maxpadlen, padding = 'post')
test_padded=pad_sequences(list_tokenized_test, maxlen=maxpadlen, padding = 'post')

In [ ]:
embedding_dim_fasttext = embedding_dim
embeddings_index_fasttext = {}
f = open('../wiki-news-300d-1M.vec', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index_fasttext[word] = np.asarray(values[1:], dtype='float32')
f.close()
embedding_matrix_fasttext = np.random.random((len(filtered_word_index) + 1, embedding_dim_fasttext))
for word, i in filtered_word_index.items():
    embedding_vector = embeddings_index_fasttext.get(word)
    if embedding_vector is not None:
        embedding_matrix_fasttext[i] = embedding_vector

In [ ]:
train_y=np.array(train_y_all['y'])
val_y=np.array(val_y_all['y'])

In [ ]:
with open('./processed/train_padded.npy', 'wb') as f:
    np.save(f, train_padded)
with open('./processed/val_padded.npy', 'wb') as f:
    np.save(f, val_padded)
with open('./processed/test_padded.npy', 'wb') as f:
    np.save(f, test_padded)
with open('./processed/train_y.npy', 'wb') as f:
    np.save(f, train_y)
with open('./processed/val_y.npy', 'wb') as f:
    np.save(f, val_y)
with open('./processed/word_index.json', 'w') as f:
    json.dump(filtered_word_index, f)
with open('./processed/embedding_matrix_fasttext.npy', 'wb') as f:
    np.save(f, embedding_matrix_fasttext)